In [19]:
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment
import cv2
import time
import json

In [5]:
with open('results\\weti1_2024_11_14_12_57_m_sam21.json', 'r') as f:
    data_mask = json.load(f)
with open('results\\weti1_2024_11_14_12_57_sam21.json', 'r') as f:
    data__no_mask = json.load(f)
with open('results\\weti1_2024_11_14_12_57.json', 'r') as f:
    data_db = json.load(f)

In [50]:
def transform_to_polygons(data, mode="points"):
    if mode == "points":
        return [points2polygon(d) for d in data]
    elif mode == "corners":
        return [corners2polygon(d) for d in data]
    elif mode == "mask":
        return [mask2polygon(d) for d in data]
    
def corners2polygon(corners):
    x1, y1, x2, y2 = corners
    points = [(x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)]
    
    return Polygon(points)
        
def points2polygon(points):
    return Polygon(points)

def mask2polygon(mask):
    mask = np.array(mask)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    largest_contour = max(contours, key=cv2.contourArea)
    
    polygon = Polygon(shell=largest_contour.squeeze())
    
    if not polygon.is_valid:
        polygon = polygon.buffer(0)
    
    return polygon

def iou(polygon1, polygon2):
    union = polygon1.union(polygon2).area
    if union <= 0:
        return 0
    intersection = polygon1.intersection(polygon2).area
    return intersection / union

def build_similarity_matrix(objects_db, objects_pred):
    len_db, len_pred = len(objects_db), len(objects_pred)
    print(f"Matrix size: [{len_db}, {len_pred}]")
    similarity_matrix = np.zeros([len_db, len_pred])
    
    time_start = time.time()
    for x, obj_db in enumerate(objects_db):
        for y, obj_pred in enumerate(objects_pred):
            similarity_matrix[x, y] = iou(obj_db, obj_pred)
            print(f"Building similarity matrix: {x * len_pred + y + 1:>10}/{len_db*len_pred}", end='\r')
    time_end = time.time() - time_start
    print(f"Similarity matrix built in {time_end:.2f}s" + ' ' * 100)
    return similarity_matrix

def match_polygons(similarity_matrix):
    xs, ys = linear_sum_assignment(similarity_matrix, maximize=True)
    matching = [[x, y, similarity_matrix[x, y]] for x, y in zip(xs, ys) if similarity_matrix[x][y] > 0]
    return matching

def evaluate_segmentation(polygons_db, polygons_pred, matching, time, weather_info = [], model_info = []):
    results = {
        "weather": weather_info,
        "model": model_info,
        "time": time
    }
    num_db = len(polygons_db)
    num_pred = len(polygons_pred)
    num_matching = len(matching)

    results['ground_truth_objects'] = num_db
    results['segmented_objects'] = num_pred
    results['matched_objects'] = num_matching

    results['precision'] = num_matching / num_pred if num_pred != 0 else 0

    results['recall'] = num_matching / num_db if num_db != 0 else 0

    results['f1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall']) if results['precision'] + results['recall'] != 0 else 0

    ious = []
    for x, y, _ in matching:
        ious.append(iou(polygons_db[x], polygons_pred[y]))

    if num_matching != 0:
        results['mean_iou'] = sum(ious) / num_matching
    else:
        results['mean_iou'] = 0

    return results

In [51]:
data_db['polygons'] = transform_to_polygons([obj['points'] for obj in data_db['objects']])
data_mask['polygons'] = transform_to_polygons([obj['segmentation'] for obj in data_mask['objects']], 'mask')

sm = build_similarity_matrix(data_db['polygons'], data_mask['polygons'])

Matrix size: [14, 12]
Similarity matrix built in 0.01s                                                                                                    


In [53]:
matching = match_polygons(sm)
print(matching)

[[1, 6, 0.9035901138065467], [4, 8, 0.9152937026458893], [6, 3, 0.9305852130628176], [7, 0, 0.9577595117489496], [9, 10, 0.9023491217868566], [10, 7, 0.8862573137732398], [11, 5, 0.9252189290968909]]


In [43]:
for p in data_db['polygons']:
    print(p)

POLYGON ((447.9481188204494 995.1189518518506, 457.58529672659597 992.7927364951946, 461.9054109603858 987.4756728228379, 464.56394279656416 979.5000773143028, 468.2194240713094 936.6312514559268, 467.55479111226487 918.6861615617228, 467.22247463274255 899.4118057494296, 460.90846152181894 887.448412486627, 452.26823305423926 878.1435510600028, 449.2773847385386 877.1466016214358, 432.66156076242385 877.8112345804805, 432.3292442829015 880.4697664166588, 447.9481188204494 995.1189518518506))
POLYGON ((431.1470976396011 876.1715407770098, 433.66024101598845 877.5423462550393, 451.4807122303715 877.3138786753677, 454.22232318643046 880.2839572110981, 468.1588455463967 880.5124247907697, 482.095367906363 877.9992814143824, 485.29391402176503 854.9240558675531, 485.29391402176503 822.7101271338606, 487.8070573981524 821.3393216558311, 488.03552497782397 819.0546458591153, 479.81069210964716 812.8860212079827, 462.2186884749357 811.9721508892965, 446.22595789792524 813.3429563673259, 434.3

In [44]:
for p in data_mask['polygons']:
    print(p)

POLYGON ((447 530, 446 531, 443 531, 442 532, 441 532, 440 533, 439 533, 438 534, 437 534, 436 535, 436 536, 433 539, 433 540, 432 541, 432 542, 431 543, 431 544, 430 545, 430 548, 429 549, 429 552, 428 553, 428 555, 427 556, 427 557, 426 558, 426 560, 425 561, 425 562, 424 563, 424 570, 423 571, 423 579, 424 580, 424 585, 425 586, 425 588, 426 589, 426 590, 428 592, 429 592, 430 593, 458 593, 459 592, 463 592, 464 591, 465 591, 466 590, 468 590, 470 588, 470 587, 471 586, 471 583, 472 582, 472 574, 473 573, 473 570, 474 569, 474 563, 475 562, 475 546, 474 545, 474 543, 473 542, 473 541, 471 539, 471 538, 469 536, 469 535, 467 533, 466 533, 465 532, 464 532, 463 531, 461 531, 460 530, 447 530))
POLYGON ((573 743, 573 741, 575 739, 596 739, 597 738, 600 738, 601 737, 604 737, 605 736, 610 736, 611 735, 618 735, 619 734, 621 734, 622 733, 624 733, 625 732, 629 732, 630 731, 632 731, 633 730, 640 730, 643 727, 645 727, 646 726, 648 726, 649 725, 650 725, 652 723, 653 723, 654 722, 655 722

In [48]:
xs, ys = linear_sum_assignment(sm, maximize=True)

for x, y in zip(xs, ys):
    if sm[x, y] > 0:
        print(x, y, sm[x, y], data_db['polygons'][x].centroid, data_mask['polygons'][y].centroid)

1 6 0.9035901138065467 POINT (457.55141252111036 845.7697574604235) POINT (459.9282253317031 845.7412942082145)
4 8 0.9152937026458893 POINT (490.6161041892949 654.4650506341472) POINT (491.1212364154641 653.5658293247818)
6 3 0.9305852130628176 POINT (510.9320200460202 557.09965730552) POINT (511.012243012243 557.5355740355741)
7 0 0.9577595117489496 POINT (449.76100511202566 563.7098616325411) POINT (449.76414241821794 563.0720525332852)
9 10 0.9023491217868566 POINT (549.9676882320615 398.41930668744135) POINT (551.0218923376818 398.5774478932374)
10 7 0.8862573137732398 POINT (506.0853041580588 402.71265303525) POINT (505.07693545791085 403.713182037663)
11 5 0.9252189290968909 POINT (458.9936948573061 403.02231518718304) POINT (459.3675509419454 403.5678585159554)
